# Transfer Learning Model for Melanoma Skin Cancer Classification

This notebook implements a transfer learning model using the ResNet50 architecture for the classification of melanoma skin cancer. The model is trained on the Melanoma Skin Cancer Dataset, which consists of 9,600 images for training and 1,000 images for evaluation. ResNet50 is chosen as the base model due to its proven performance in image classification tasks.

## Dataset Overview

The Melanoma Skin Cancer Dataset contains a total of 10,000 images, with 9,600 images for training and 1,000 images for evaluation. The dataset is designed to facilitate the development of deep learning models for accurate classification of melanoma, which is a deadly form of skin cancer. Early detection and treatment of melanoma can significantly improve patient outcomes and save lives.

## Model Architecture

The transfer learning model utilizes the ResNet50 architecture as the base model. ResNet50 is a deep convolutional neural network that has been pre-trained on a large-scale image dataset, such as ImageNet. By leveraging the pre-trained weights, we can take advantage of the learned feature representations and adapt them to our specific task of melanoma classification.

The pre-trained ResNet50 model is loaded, and its top layers are removed. We then add our own classification layers on top of the base model to perform binary classification (melanoma vs. non-melanoma). The weights of the pre-trained layers are initially frozen to prevent them from being updated during training, allowing us to focus on training the newly added classification layers.

## Setup
Import necessary libraries.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Data preparation:
- Split the datset into the training and validation sets.
- Preprocess the images by resizing to a consistent size and normalizing pixel values.
- Apply data augmentation techniques to increase diversity of training.

In [2]:
# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = datasets.ImageFolder(root='melanoma_cancer_dataset/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='melanoma_cancer_dataset/test', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Model Setup:
- Load the pre-trained ResNet50 model.
- Remove teh top layers of teh pre-trained model.
- Add new classification layeres specific to the binary classification task.

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 model
model = models.resnet18(pretrained=True)

# Replace the last fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: benign and malignant

# Transfer the model to the device
model = model.to(device)

/Users/adityashrey/opt/anaconda3/envs/cs4262/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/adityashrey/opt/anaconda3/envs/cs4262/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/adityashrey/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


## Model Training 
Freeze the weights of the pre-trained model. Train the model on preprocessed training data using binary cross entropy. Monitor the trainign progress and validation performance.

In [4]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    train_loss = train_loss / len(train_dataset)


## Model Evaluation
Evaluate the trained model on held-out evaluation set. Asses the model's performance.

In [6]:
from sklearn.metrics import f1_score

model.eval()
val_loss = 0.0
val_accuracy = 0.0
val_preds = []
val_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        val_loss += loss.item() * images.size(0)
        
        _, predicted = torch.max(outputs, 1)
        val_accuracy += torch.sum(predicted == labels).item()

        val_preds.extend(predicted.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

val_loss = val_loss / len(val_dataset)
val_accuracy = val_accuracy / len(val_dataset)

val_f1 = f1_score(val_labels, val_preds, average='weighted')

print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1:.4f}")

Epoch [10/10], Train Loss: 0.1762, Val Loss: 0.2550, Val Accuracy: 0.8990, Val F1 Score: 0.8986
